In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import warnings
import numpy as np
import matplotlib.pyplot as plt
from xgboost import plot_importance
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

y1

第四问

In [ ]:
demo = pd.read_feather('classes_DB_IQR.feather')
class_ = demo['class']
features = demo.drop(columns=['class'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(features)
y = pd.Series(class_)
y = y-1

In [ ]:
from xgboost import XGBClassifier
xgb_param_grid = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.04, 0.05, 0.06],
    'n_estimators': [150, 200, 250],
    'min_child_weight': [2, 3, 4],
    'scale_pos_weight': [0.2, 0.5, 0.8],
    'subsample': [0.8, 0.9, 1],  
    'colsample_bytree': [0.3, 0.5, 0.8],
    'colsample_bylevel': [0.7, 0.8, 0.9], 
    'reg_alpha': [0.01, 0.05, 0.1],  
    'reg_lambda': [0.05, 0.1, 0.2], 
    'max_delta_step': [1, 2, 3],    
    'gamma': [0, 0.1, 0.2]
}
xgb = XGBClassifier(random_state=0)

In [ ]:
# 数据集划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Initialize the XGBoost Classifier using optimal hyperparameters
xgb_opt = XGBClassifier(max_depth=6,
                        learning_rate=0.05,
                        n_estimators=200,  
                        min_child_weight=2,
                        scale_pos_weight=0.5,
                        subsample=0.9 ,
                        colsample_bytree=0.5,
                        colsample_bylevel=0.8 ,
                        reg_alpha=0.05 ,
                        reg_lambda=0.1 ,
                        max_delta_step=2 ,
                        gamma=0.1,
                        random_state=0)

# Train the XGBoost classifier
xgb_opt.fit(X_train, y_train)

In [ ]:
# 输出准确率
train_acc = accuracy_score(y_train, xgb_opt.predict(X_train))
print(f"The accuracy score is {train_acc:.5f}")
test_acc = accuracy_score(y_test, xgb_opt.predict(X_test))
print(f"The accuracy score is {test_acc:.5f}")

In [ ]:
# 可视化特征重要性
plt.figure(figsize=(10, 8))
plot_importance(xgb_opt, importance_type='weight')  # 你可以使用 'weight', 'gain', 'cover' 三种不同的指标
plt.title('Feature Importance')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 计算并可视化混淆矩阵
cm = confusion_matrix(y_test, xgb_opt.predict(X_test))
cmd = ConfusionMatrixDisplay(cm, display_labels=xgb_opt.classes_)

plt.figure(figsize=(8, 8))
cmd.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# 获取特征重要性并排序
importance_dict = xgb_opt.get_booster().get_score(importance_type='weight')
sorted_importance = sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)

# 获取前10个特征
top_10_features = sorted_importance[:10]
top_10_names = [feature for feature, importance in top_10_features]
top_10_importance = [importance for feature, importance in top_10_features]
print(top_10_features)